In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto(
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8),
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [2]:
import amp.data_utils.data_loader as data_loader
import amp.data_utils.sequence as du_sequence
import numpy as np
import pandas as pd
import sklearn
from amp.config import pepcvae
from amp.models.decoders import amp_expanded_decoder
from amp.models.encoders import amp_expanded_encoder
from amp.models.master import master
from amp.utils import basic_model_serializer, callback, generator
from keras import backend, layers
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

In [4]:
seed = 7
np.random.seed(seed)

In [5]:
from amp.config import MIN_LENGTH, MAX_LENGTH, LATENT_DIM, MIN_KL, RCL_WEIGHT, HIDDEN_DIM, MAX_TEMPERATURE

kl_weight = backend.variable(MIN_KL, name="kl_weight")
tau = backend.variable(MAX_TEMPERATURE, name="temperature")

# Import data

In [6]:
data_manager = data_loader.AMPDataManager(
    '../data/unlabelled_positive.csv',
    '../data/unlabelled_negative.csv',
    min_len=MIN_LENGTH,
    max_len=MAX_LENGTH)

amp_x, amp_y = data_manager.get_merged_data()
amp_x_train, amp_x_test, amp_y_train, amp_y_test = train_test_split(amp_x, amp_y, test_size=0.1, random_state=36)
amp_x_train, amp_x_val, amp_y_train, amp_y_val = train_test_split(amp_x_train, amp_y_train, test_size=0.2, random_state=36)

/home/pszymczak/amp/new-tf/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/pszymczak/amp/new-tf/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [7]:
# Restrict the length
ecoli_df = pd.read_csv('../data/mic_data.csv')
mask = (ecoli_df['sequence'].str.len() <= MAX_LENGTH) & (ecoli_df['sequence'].str.len() >= MIN_LENGTH)
ecoli_df = ecoli_df.loc[mask]
mic_x = du_sequence.pad(du_sequence.to_one_hot(ecoli_df['sequence']))
mic_y = ecoli_df.value

In [8]:
mic_x_train, mic_x_test, mic_y_train, mic_y_test = train_test_split(mic_x, mic_y, test_size=0.1, random_state=36)
mic_x_train, mic_x_val, mic_y_train, mic_y_val = train_test_split(mic_x_train, mic_y_train, test_size=0.2, random_state=36)

In [9]:
new_train = pd.DataFrame()
new_train['value'] = list(mic_y_train)
new_train['sequence'] = list(mic_x_train)
active_peptides = new_train[new_train.value < 1.5]
active_peptides  = pd.concat([active_peptides] * 25)
new_train  = pd.concat([
    new_train,
    active_peptides,    
])
mic_x_train = np.array(list(new_train.sequence)).reshape(len(new_train), 25)
mic_y_train = np.array(new_train.value).reshape(len(new_train),)

# Import pretrained classifier and regressor

In [10]:
bms = basic_model_serializer.BasicModelSerializer()
amp_classifier = bms.load_model('../models/amp_classifier')
amp_classifier_model = amp_classifier()
mic_classifier = bms.load_model('../models/mic_classifier/')
mic_classifier_model = mic_classifier() 

# Set up cVAE

In [11]:
encoder = amp_expanded_encoder.AMPEncoderFactory.get_default(HIDDEN_DIM, LATENT_DIM, MAX_LENGTH)
input_to_encoder = layers.Input(shape=(MAX_LENGTH,))
encoder_model = encoder(input_to_encoder)
decoder = amp_expanded_decoder.AMPDecoderFactory.build_default(LATENT_DIM + 2, tau, MAX_LENGTH)
input_to_decoder = layers.Input(shape=(LATENT_DIM + 2,))
decoder_model = decoder(input_to_decoder)

tracking <tf.Variable 'temperature:0' shape=() dtype=float32, numpy=2.0> temperature


In [12]:
master_model = master.MasterAMPTrainer(
    amp_classifier=amp_classifier,
    mic_classifier=mic_classifier,
    encoder=encoder,
    decoder=decoder,
    kl_weight=kl_weight,
    rcl_weight=RCL_WEIGHT, 
    master_optimizer=Adam(lr=1e-3),
    loss_weights=pepcvae,
)

In [13]:
master_keras_model = master_model.build(input_shape=(MAX_LENGTH, 21))

tracking <tf.Variable 'kl_weight:0' shape=() dtype=float32, numpy=1e-04> kl_weight
Tensor("vae_loss_1/add_1:0", shape=(None,), dtype=float32)


In [14]:
vae_callback = callback.VAECallback(
    encoder=encoder_model,
    decoder=decoder_model,
    tau=tau,
    kl_weight=kl_weight,
    amp_classifier=amp_classifier_model,
    mic_classifier=mic_classifier_model,
)


sm_callback = callback.SaveModelCallback(
    model = master_model,
    model_save_path="../models/final_models/",
    name="PepCVAE")

In [15]:
#DATASET_AMP/MIC_TRAIN/VAL

amp_amp_train = amp_classifier_model.predict(amp_x_train, verbose=1, batch_size=10000).reshape(len(amp_x_train))
amp_mic_train = mic_classifier_model.predict(amp_x_train, verbose=1, batch_size=10000).reshape(len(amp_x_train))
amp_amp_val = amp_classifier_model.predict(amp_x_val, verbose=1, batch_size=10000).reshape(len(amp_x_val))
amp_mic_val = mic_classifier_model.predict(amp_x_val, verbose=1, batch_size=10000).reshape(len(amp_x_val))

mic_amp_train = amp_classifier_model.predict(mic_x_train, verbose=1, batch_size=10000).reshape(len(mic_x_train))
mic_mic_train = mic_classifier_model.predict(mic_x_train, verbose=1, batch_size=10000).reshape(len(mic_x_train))
mic_amp_val = amp_classifier_model.predict(mic_x_val, verbose=1, batch_size=10000).reshape(len(mic_x_val))
mic_mic_val = mic_classifier_model.predict(mic_x_val, verbose=1, batch_size=10000).reshape(len(mic_x_val))

uniprot_x_train = np.array(du_sequence.pad(du_sequence.to_one_hot(pd.read_csv('../data/Uniprot_0_25_train.csv').Sequence)))
uniprot_x_val = np.array(du_sequence.pad(du_sequence.to_one_hot(pd.read_csv('../data/Uniprot_0_25_val.csv').Sequence)))

uniprot_amp_train = amp_classifier_model.predict(uniprot_x_train, verbose=1, batch_size=10000).reshape(len(uniprot_x_train))
uniprot_mic_train = mic_classifier_model.predict(uniprot_x_train, verbose=1, batch_size=10000).reshape(len(uniprot_x_train))
uniprot_amp_val = amp_classifier_model.predict(uniprot_x_val, verbose=1, batch_size=10000).reshape(len(uniprot_x_val))
uniprot_mic_val = mic_classifier_model.predict(uniprot_x_val, verbose=1, batch_size=10000).reshape(len(uniprot_x_val))


22524/22524 [==============================] - 0s 2us/step


In [16]:
training_generator = generator.concatenated_generator(
    uniprot_x_train,
    uniprot_amp_train,
    uniprot_mic_train,
    amp_x_train,
    amp_amp_train,
    amp_mic_train,
    mic_x_train,
    mic_amp_train,
    mic_mic_train,
    128
)
validation_generator = generator.concatenated_generator(
    uniprot_x_val,
    uniprot_amp_val,
    uniprot_mic_val,
    amp_x_val,
    amp_amp_val,
    amp_mic_val,
    mic_x_val,
    mic_amp_val,
    mic_mic_val,
    128
)

In [ ]:
history = master_keras_model.fit_generator(
    training_generator,
    steps_per_epoch=1408,
    epochs=60,
    validation_data=validation_generator,
    validation_steps=176,
    callbacks=[vae_callback, sm_callback],
)


/home/pszymczak/amp/new-tf/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Epoch 1/30
